# QMUL Careers Team Python Script
### A detailed guide on how to run this programme.



Each text box describe what the code does in the cell below it.

The first cell imports some of the software that the programme needs, if you click the small play button above, or press Shift and Enter, it will run the cell.

In [ ]:
import pandas as pd
import openpyxl
import numpy as np
import os
import ipywidgets as widgets

We need the excel workbook that the programme will use, and within that the excel spreadsheet with the student information. Please name the spreadsheet with the application information: "data" and save it on your computer.

The video below shows how to add this to the programme.

In [ ]:
from IPython.display import Video; from ipywidgets import interactive, IntSlider

vid = Video(filename="media/add_applications.mov",data="", width=800, height = 600)
display(vid)



Next, point the programme at the excel sheet just uploaded.

In [ ]:
def get_sheet():
    for file in os.listdir('.'):
        if file.endswith(".xlsx"):
            return file

filename = get_sheet()

Read the spreadsheet in, and perform some data cleaning.

In [ ]:
df = pd.read_excel(filename,"data")


Any missing scores will be replaced with the average score of the dataset.

In [ ]:
df['Total score'].fillna(df['Total score'].mean(), inplace=True)

Some exploratory data analysis in the next 2 cells

In [ ]:
df['Gender'].value_counts()

In [ ]:
df['YearOfStudy'].value_counts()

Take only the columns important to establishing the groups. 

In [ ]:
group_df = df[[
'email', 
'Course', 
'YearOfStudy',
'Gender', 
'Bursary holder', 
'Client meetings delivery', 
'Total score',
'Personality type']]

'email', 
'Course', 
'YearOfStudy',
'Gender', 
'Bursary holder', 
'Client meetings delivery', 
'Total score',
'Personality type'

In [ ]:
group_df = group_df.assign(groupno='')


You can decide which variables you would like to prioritize your sort by.

In [ ]:
the_vars = widgets.SelectMultiple(
    options=[
'Course', 
'YearOfStudy',
'Gender', 
'Bursary holder', 
'Client meetings delivery', 
'Personality type'],
    value=['Gender'],
    #rows=10,
    description='Scores',
    disabled=False
)
the_vars

The script will always prioritise total score first and foremost.

In [ ]:
to_sort = list(the_vars.value)
to_sort.insert(0,'Total score')


#### The Sorting Algorithm

In [ ]:
def assign_group(df):
    df.sort_values(by=to_sort, ascending=False, inplace=True)
    count = 1
    up = True
    for i,row in group_df.iterrows():

        if count <= round(len(df)/6,0) and up == True:
            df.at[i,'groupno'] = count
            count += 1

        elif count > round(len(df)/6,0) and up == True:
            df.at[i,'groupno'] = count-1
            count-=1
            up = False

        if count > 1 and up == False:
            df.at[i,'groupno'] = count
            count -= 1
        elif count == 1 and up == False:
            df.at[i,'groupno'] = count
            up = True
    
    return df

assigned = assign_group(group_df)

    

More data analysis for the different groups

In [ ]:
assigned.groupby('groupno').mean()

In [ ]:
assigned.pivot_table(index=['groupno'], columns=['YearOfStudy'], aggfunc='size', fill_value=0)


In [ ]:
assigned.pivot_table(index=['groupno'], columns=['Gender'], aggfunc='size', fill_value=0)


### Send the groups back into the original spreadsheet

In [ ]:
subset = assigned[['email','groupno']]

final = pd.merge(df,subset, how='inner', on='email')



Write them into the spreadsheet.

In [ ]:
book = openpyxl.load_workbook(filename)
writer = pd.ExcelWriter(filename, engine = 'openpyxl')
writer.book = book
final.to_excel(writer, sheet_name='New Sheet Groups')
writer.save()
writer.close()

## Make sure you download your new spreadsheet, you can copy and paste it into your previous working document.